## Duolingo en->ja dataset translation with Google Translate

In [1]:
# Import packages
from googletrans import Translator # Unofficial Google Translate API
from janome.tokenizer import Tokenizer # Tokenizer for Japanese sentences
import nltk
from nltk.translate.bleu_score import sentence_bleu # BLEU translation scoring algorithm

# Ensure NLTK is downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Data reading

In [2]:
# Read the Duolingo en->ja train text file
with open('Data/Duolingo/en_ja/train.en_ja.2020-01-13.gold.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [3]:
# Create a list with all the source langugage (English) sentences

# Initialize variables
english_sentences = []
current_sentence = None

# Iterate through lines
for line in lines:
    line = line.strip()
    if line.startswith("prompt_"):
        # Extract English sentence from the first line
        current_sentence = line.split("|")[1]
        # Append English sentence to the list
        english_sentences.append(current_sentence)
    else:
        continue

In [23]:
# Print the first element of the English sentences list
english_sentences[0:4]

['she is just a child.',
 'the victim is a citizen.',
 'they are soldiers.',
 'this scene is beautiful.']

In [5]:
# Create a list with all the target language (Japanese) reference sentences

# Initialize variables
japanese_sentences = []
current_group = []

# Iterate through lines
for line in lines:
    line = line.strip()
    if line.startswith("prompt_"):
        # If a new group starts, append the current group to the japanese_sentences list
        if current_group:
            japanese_sentences.append(current_group)
            current_group = []
    elif line == "":
        # If an empty line is encountered, move to the next group
        continue
    else:
        # Append the Japanese sentence to the current group
        current_group.append(line.split("|")[0])

# Append the last group to the japanese_sentences list
if current_group:
    japanese_sentences.append(current_group)

In [6]:
# Print the first element of the Japanese reference sentences list
japanese_sentences[0]

['彼女はただの子供です。',
 '彼女はまだ子供です。',
 '彼女はほんの子供です。',
 '彼女はただの子供だ。',
 '彼女はほんの子どもです。',
 '彼女は単なる子供です。',
 '彼女はほんの子供だ。',
 '彼女は子供です。',
 '彼女はただの子どもです。',
 '彼女はただの女の子です。',
 '彼女は単なる小さい子に過ぎません。',
 '彼女はまだ女の子。',
 '彼女は単に小さい子に過ぎない。',
 '彼女は子供。',
 '彼女は子どもに過ぎない。',
 '彼女はこどもだよ。',
 '彼女はほんの子供に過ぎない。',
 '彼女はほんの子。',
 '彼女はまだ子。',
 '彼女はただのこども。',
 '彼女は単なる子どもである。',
 '彼女はただの子供に過ぎない。',
 '彼女はまだ子供に過ぎません。',
 '彼女は単なる子です。',
 '彼女はまだ子供に過ぎない。',
 '彼女はただの小さい子に過ぎない。',
 '彼女はただの小さい子。',
 '彼女は単に子に過ぎない。',
 '彼女はただの小さい子だよ。',
 '彼女はただのこどもだよ。',
 '彼女はまだ子どもです。',
 '彼女は単なる小さい子です。',
 '彼女は子ども。',
 '彼女はまだこどもだよ。',
 '彼女は単なる子に過ぎません。',
 '彼女はまだ女の子に過ぎない。',
 '彼女は単にこどもだ。',
 '彼女はこどもに過ぎない。',
 '彼女は単にこどもに過ぎません。',
 '彼女は単なるこどもです。',
 '彼女はまだ子ども。',
 '彼女は単なる女の子だよ。',
 '彼女は子に過ぎません。',
 '彼女は単に子どもだ。',
 '彼女はほんの小さい子に過ぎない。',
 '彼女はただのこどもである。',
 '彼女はほんのこども。',
 '彼女はただの子である。',
 '彼女はほんの子だ。',
 '彼女はまだ小さい子に過ぎません。',
 '彼女は単なる子。',
 '彼女はほんの子ども。',
 '彼女はほんの女の子だよ。',
 '彼女は子。',
 '彼女はまだ子どもに過ぎません。',
 '彼女は単にこども。',
 '彼女はまだ女の子だよ。',
 '彼女は子供に過ぎません。',
 '彼女はまだこどもです。',
 '彼女はほんの子どもだよ。',
 '彼女はただの小さい子である。'

### Translation

In [7]:
# Initialize Googletrans Translator
translator = Translator()

In [8]:
# Initialize dictionary to store translated conversations
candidate_sentences = []

# Translate the first 4 english sentences
for sentence in english_sentences[0:4]:
    translation = translator.translate(sentence, src='en', dest='ja').text
    candidate_sentences.append(translation)

In [12]:
# Print candidates
candidate_sentences

['彼女はただの子供です。', '犠牲者は市民です。', '彼らは兵士です。', 'このシーンは美しいです。']

### Tokenize sentences

In [13]:
# Initialize tokenizer
tokenizer = Tokenizer()

In [14]:
# List to store tokenized candidates
tokenized_candidates = []
# Tokenize each candidate sentence and store the tokenized forms
for sentence in candidate_sentences:
    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)
    # Extract surface forms (actual words) from tokens
    tokenized_form = [token.surface for token in tokens]
    # Append tokenized form to the list
    tokenized_candidates.append(tokenized_form)

In [15]:
# Print the first element of tokenized candidates
tokenized_candidates[0]

['彼女', 'は', 'ただ', 'の', '子供', 'です', '。']

In [16]:
# List to store tokenized references
tokenized_references = []
# Tokenize each sentence in each inner list and store the tokenized forms
for sentence_group in japanese_sentences[0:4]:
    tokenized_group = []
    for sentence in sentence_group:
        # Tokenize the sentence
        tokens = tokenizer.tokenize(sentence)
        # Extract surface forms (actual words) from tokens
        tokenized_form = [token.surface for token in tokens]
        # Append tokenized form to the group
        tokenized_group.append(tokenized_form)
    # Append tokenized group to the list
    tokenized_references.append(tokenized_group)

In [17]:
# Print the first element of tokenized references
tokenized_references[0]

[['彼女', 'は', 'ただ', 'の', '子供', 'です', '。'],
 ['彼女', 'は', 'まだ', '子供', 'です', '。'],
 ['彼女', 'は', 'ほんの', '子供', 'です', '。'],
 ['彼女', 'は', 'ただ', 'の', '子供', 'だ', '。'],
 ['彼女', 'は', 'ほんの', '子ども', 'です', '。'],
 ['彼女', 'は', '単なる', '子供', 'です', '。'],
 ['彼女', 'は', 'ほんの', '子供', 'だ', '。'],
 ['彼女', 'は', '子供', 'です', '。'],
 ['彼女', 'は', 'ただ', 'の', '子ども', 'です', '。'],
 ['彼女', 'は', 'ただ', 'の', '女の子', 'です', '。'],
 ['彼女', 'は', '単なる', '小さい', '子', 'に', '過ぎ', 'ませ', 'ん', '。'],
 ['彼女', 'は', 'まだ', '女の子', '。'],
 ['彼女', 'は', '単に', '小さい', '子', 'に', '過ぎ', 'ない', '。'],
 ['彼女', 'は', '子供', '。'],
 ['彼女', 'は', '子ども', 'に', '過ぎ', 'ない', '。'],
 ['彼女', 'は', 'こども', 'だ', 'よ', '。'],
 ['彼女', 'は', 'ほんの', '子供', 'に', '過ぎ', 'ない', '。'],
 ['彼女', 'は', 'ほんの', '子', '。'],
 ['彼女', 'は', 'まだ', '子', '。'],
 ['彼女', 'は', 'ただ', 'の', 'こども', '。'],
 ['彼女', 'は', '単なる', '子ども', 'で', 'ある', '。'],
 ['彼女', 'は', 'ただ', 'の', '子供', 'に', '過ぎ', 'ない', '。'],
 ['彼女', 'は', 'まだ', '子供', 'に', '過ぎ', 'ませ', 'ん', '。'],
 ['彼女', 'は', '単なる', '子', 'です', '。'],
 ['彼女', 'は', 'まだ', '子供', 'に

### Translation scoring

In [21]:
# Initialize list to store BLEU scores
scores_duolingo = []

# Calculate BLEU scores for the first 4 sentences
for i in range(4):
    score = sentence_bleu(tokenized_references[i], tokenized_candidates[i])       # Calculate BLEU score
    scores_duolingo.append(score)

In [22]:
# Print scores
scores_duolingo

[1.0, 1.0, 1.0, 1.0]